# Mount Drive



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Importing Libraries and initializing stopwords and stemmer


In [ ]:
!pip3 install texthero
!pip3 install transformers
!pip3 install tensorflow_addons


In [ ]:
!pip install tweet-preprocessor

import seaborn as sns 
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
import re 
import nltk
from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer 
from textblob import TextBlob,Word
from nltk.corpus import words
nltk.download('words')
nltk.download('wordnet')
import texthero as hero
import re
from texthero import stopwords

from nltk.corpus import wordnet

import tensorflow as tf

from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

import tensorflow as tf

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix

In [ ]:
def lemma_per_pos(sent):
    '''function to lemmatize according to part of speech tag'''
    tweet_tokenizer=TweetTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    lemmatized_list = [lemmatizer.lemmatize(w) for w in  tweet_tokenizer.tokenize(sent)]
    return " ".join(lemmatized_list)

def df_preprocessing(df,feature_col):
    '''
    Preprocessing of dataframe
    '''
    stop = set(stopwords.words('english'))
    df[feature_col]= (df[feature_col].pipe(hero.lowercase).
                      pipe(hero.remove_urls).
                      pipe(hero.remove_digits).
                      pipe(hero.remove_punctuation).
                      pipe(hero.remove_html_tags) )
    # lemmatization
    df[feature_col]= [lemma_per_pos(sent) for sent in df[feature_col]]
    #df[col_name]= hero.remove_stopwords(df[col_name],custom_stopwords)
    return df

In [ ]:
from transformers import AutoTokenizer,TFDistilBertModel, DistilBertConfig
from transformers import TFAutoModel
import tensorflow as tf 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from transformers import AdamW, get_linear_schedule_with_warmup
import tensorflow_addons as tfa

# Reading Data


In [ ]:
hasoc_data = pd.read_csv("drive/MyDrive/Dataset/Hasoc2021_train.csv") #Your respective address
hasoc_data.head()


In [ ]:
aakash_test = pd.read_csv("drive/MyDrive/Dataset/aakash_300.csv")

In [ ]:
aakash_test.describe()

In [ ]:
aakash_test.head()

In [ ]:
test_hasoc = pd.read_csv("drive/MyDrive/Dataset/en_Hasoc2021_test_task1.csv")
test_hasoc.head()

In [ ]:
hasoc_data.describe()

In [ ]:
test_hasoc_2 = pd.read_csv("drive/MyDrive/Dataset/en_Hasoc2021_test_task2.csv")
test_hasoc_2.head()

In [ ]:
#reading the data
data= pd.read_csv('drive/MyDrive/Dataset/modified_hasoc_task21.csv')
train_df = data
# test_df = aakash_test
#test_df= pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')   TEST DATA TO BE PROVIDED
print(train_df.columns,train_df.shape)
target_col= train_df.columns[2:]
feature_col= train_df.columns[1:2]
train_df.head()

In [ ]:
column_names = ["id","text","hate","offen","prof","none"]
test_df = pd.DataFrame(columns = column_names)


In [ ]:
for i in range(len(aakash_test)):
  id_temp = aakash_test['_id'].iloc[i]
  text_temp = aakash_test['text'].iloc[i]
  if aakash_test['task_2'].iloc[i] == "NONE":
    test_df.loc[i] = [id_temp,text_temp,0,0,0,1]
  elif aakash_test['task_2'].iloc[i] == "PRFN":
    test_df.loc[i] = [id_temp,text_temp,0,0,1,0]
  elif aakash_test['task_2'].iloc[i] == "OFFN":
    test_df.loc[i] = [id_temp,text_temp,0,1,0,0]
  elif aakash_test['task_2'].iloc[i] == "HATE":
    test_df.loc[i] = [id_temp,text_temp,1,0,0,0]

In [ ]:
test_df

#Preprocessing

In [ ]:
 target_col,feature_col

In [ ]:
with tf.device('/GPU:0'): 
 proc_train_df= df_preprocessing(train_df,feature_col[0])

In [ ]:
proc_test_df = df_preprocessing(test_df,feature_col[0])

In [ ]:
#Creating tokenizer
def create_tokenizer(pretrained_weights='distilbert-base-uncased'):
  '''Function to create the tokenizer'''

  tokenizer = AutoTokenizer.from_pretrained(pretrained_weights)
  return tokenizer

#Tokenization of the data
def data_tokenization(dataset,feature_col,max_len,tokenizer):
    '''dataset: Pandas dataframe with feature name is column name 
    Pretrained_weights: selected model 
    RETURN: [input_ids, attention_mask]'''

    tokens = dataset[feature_col].apply(lambda x: tokenizer(x,return_tensors='tf', 
                                                            truncation=True,
                                                            padding='max_length',
                                                            max_length=max_len, 
                                                            add_special_tokens=True))
    input_ids= []
    attention_mask=[]
    for item in tokens:
        input_ids.append(item['input_ids'])
        attention_mask.append(item['attention_mask'])
    input_ids, attention_mask=np.squeeze(input_ids), np.squeeze(attention_mask)


    return [input_ids,attention_mask]

#Model 

In [ ]:
def bert_model(pretrained_weights,max_len,learning_rate):
  '''BERT model creation with pretrained weights
  INPUT:
  pretrained_weights: Language model pretrained weights
  max_len: input length '''
  print('Model selected:', pretrained_weights)
  bert=TFAutoModel.from_pretrained(pretrained_weights)
  
  # This is must if you would like to train the layers of language models too.
  for layer in bert.layers:
      layer.trainable = True

  # # parameter declaration
  # step = tf.Variable(0, trainable=False)
  # schedule = tf.optimizers.schedules.PiecewiseConstantDecay([10000, 15000], [2e-0, 2e-1, 1e-2])
  # # lr and wd can be a function or a tensor
  # lr = learning_rate * schedule(step)
  # wd = lambda:lr * schedule(step)
  # optimizer = tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd)

  optimizer= tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
#   optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)

  # declaring inputs, BERT take input_ids and attention_mask as input
  input_ids= Input(shape=(max_len,),dtype=tf.int32,name='input_ids')
  attention_mask=Input(shape=(max_len,),dtype=tf.int32,name='attention_mask')

  bert= bert(input_ids,attention_mask=attention_mask)
  x= bert[0][:,0,:]
  x=tf.keras.layers.Dropout(0.12)(x)
  # x= tf.keras.layers.Dense(128)(x)
  x=tf.keras.layers.Dense(64)(x)
  x=tf.keras.layers.Dense(32)(x)

  output=tf.keras.layers.Dense(4,activation='sigmoid')(x)

  model=Model(inputs=[input_ids,attention_mask],outputs=[output])
  # compiling model 
  model.compile(optimizer=optimizer,
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE,name='binary_crossentropy'),
                metrics=['accuracy'])
  return model

In [ ]:
pretrained_weights='bert-base-uncased'
max_len=256
epochs=15
learning_rate=2e-5
batch_size=4

In [ ]:
tokenizer= create_tokenizer(pretrained_weights)

In [ ]:
x_train= data_tokenization(proc_train_df,feature_col[0],max_len,tokenizer)

In [ ]:
x_train

In [ ]:
y_train= proc_train_df[target_col].values
y_train

In [ ]:
y_train.shape

In [ ]:
bert=bert_model(pretrained_weights,max_len,learning_rate)
bert.summary()

In [ ]:
with tf.device('/GPU:0'):
    bert.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1)

#Aakash Prediction

In [ ]:
test_ids= test_df['id']
x_test= data_tokenization(proc_test_df,feature_col[0],max_len,tokenizer)
x_test

In [ ]:
test_df['id']

In [ ]:
preds= bert.predict(x_test)



In [ ]:
aakash_df= pd.DataFrame(preds, columns= target_col)
aakash_df['id']=list(test_df['id'])
aakash_df

In [ ]:
aakash_df = aakash_df[['id']+target_col.tolist()]
aakash_df

In [ ]:
column_names = ["id", "label"]
aakash_result_df = pd.DataFrame(columns = column_names)


In [ ]:
for i in range(len(aakash_df)):
  temp = max(aakash_df['hate'].iloc[i],aakash_df['offen'].iloc[i],aakash_df['none'].iloc[i],aakash_df['prof'].iloc[i])
  id_temp = aakash_df['id'].iloc[i]
  if temp == aakash_df['hate'].iloc[i]:
    aakash_result_df.loc[i] = [id_temp,'HATE']
  elif temp == aakash_df['offen'].iloc[i]:
    aakash_result_df.loc[i] = [id_temp,'OFFN']
  elif temp == aakash_df['prof'].iloc[i]:
    aakash_result_df.loc[i] = [id_temp,'PRFN']
  elif temp == aakash_df['none'].iloc[i]:
    aakash_result_df.loc[i] = [id_temp,'NONE']


In [ ]:
aakash_result_df

In [ ]:
values = (aakash_test['task_2'] == aakash_result_df['label']).value_counts()
values

In [ ]:
accuracy = values[1]/(values[0]+values[1])
accuracy

#Submission file generation

In [ ]:
submission_ids = test_hasoc_2._id

In [ ]:
proc_subm_df = df_preprocessing(test_hasoc_2,feature_col[0])

In [ ]:
subm_data = data_tokenization(proc_subm_df,feature_col[0],max_len,tokenizer)

In [ ]:
submission_ids

In [ ]:
preds= bert.predict(subm_data)

In [ ]:
submiss_df= pd.DataFrame(preds, columns= target_col)
submiss_df['id']=list(submission_ids)
submiss_df

In [ ]:
submiss_df = submiss_df[['id']+target_col.tolist()]
submiss_df

In [ ]:
column_names = ["id", "label"]
submission_df = pd.DataFrame(columns = column_names)
submission_df

In [ ]:
for i in range(len(submiss_df)):
  temp = max(submiss_df['hate'].iloc[i],submiss_df['offen'].iloc[i],submiss_df['none'].iloc[i],submiss_df['prof'].iloc[i])
  id_temp = submiss_df['id'].iloc[i]
  if temp == submiss_df['hate'].iloc[i]:
    submission_df.loc[i] = [id_temp,'HATE']
  elif temp == submiss_df['offen'].iloc[i]:
    submission_df.loc[i] = [id_temp,'OFFN']
  elif temp == submiss_df['prof'].iloc[i]:
    submission_df.loc[i] = [id_temp,'PRFN']
  elif temp == submiss_df['none'].iloc[i]:
    submission_df.loc[i] = [id_temp,'NONE']

In [ ]:
submission_df

In [ ]:
submission_df.to_csv('drive/MyDrive/Submissions/submission4_bert_hasoc_t2.csv', index = False)
